#### Note:
**2008.csv** file is too large so it is not in my git repository. If you want to run the scripts then you may have to download the file from http://stat-computing.org/dataexpo/2009/the-data.html.

# Parsing the data
### Final Project - Data Visualization Course
### Udacity Data Analyst Nanodegree

For this final project I will use 2008 flights data that I downloaded from http://stat-computing.org/dataexpo/2009/the-data.html. The file doesn't have the geographic coordination of each airport so I downloaded the file that contains the information of coordinates of each airport from https://gist.githubusercontent.com/tdreyno/4278655/raw/7b0762c09b519f40397e4c3e100b097d861f5588/airports.json.


First I will load the json file and save it as a dictionary.

In [1]:
### import necessary libraries
import json

### Create an empty dictionary
airport_dict = {}

### load the json file
count = 0
with open("airport.json") as F:
    for i in json.load(F):
        try:
            airport_dict[i['code']] = {'lon': i['lon'], 'lat': i['lat']}
        except:
            print i['code'], ":", i['name']
            count += 1

In [2]:
print "Saved airports:",len(airport_dict)
print "Excluded airports:", count

Saved airports: 3885
Excluded airports: 0


Now we will load the flights file and parse the data.

Since there are too many records in this file I will filter the january data only.

When you look at the data in **2008.csv** the same Destination and Origin are repeated at the same day with different hours. I will include only one of them and exclude redundant data.

In [3]:
### load csv library
import csv

### List for avoiding redundancy
redun = []

### load csv data
id_count = 0
with open("2008.csv") as csvfile:
    with open("airport_coord.csv", "wb") as wfile:
        fieldnames = ['id', 'day', 
                      'Dest', 'Dest_lon', 'Dest_lat', 
                      'Origin', 'Origin_lon', 'Origin_lat']
        writer = csv.DictWriter(wfile, fieldnames=fieldnames)
        writer.writeheader()
        
        ### write rows
        for i in csv.DictReader(csvfile):
            if int(i["Month"]) == 1:
                if i['Dest'] != 'OGD':
                    tmp_dict = {}
                    tmp_dict['day'] = i["DayofMonth"]
                    tmp_dict['Dest'] = i['Dest']
                    tmp_dict['Dest_lon'] = airport_dict[i['Dest']]['lon']
                    tmp_dict['Dest_lat'] = airport_dict[i['Dest']]['lat']
                    tmp_dict['Origin'] = i['Origin']
                    tmp_dict['Origin_lon'] = airport_dict[i['Origin']]['lon']
                    tmp_dict['Origin_lat'] = airport_dict[i['Origin']]['lat']
                    
                    ### Check if the item is redundant
                    if tmp_dict['day']+tmp_dict['Dest']+tmp_dict['Origin'] not in redun:
                        redun.append(tmp_dict['day']+tmp_dict['Dest']+tmp_dict['Origin'])
                        tmp_dict['id'] = id_count

                        ### Write rows
                        writer.writerow(tmp_dict)
                        id_count += 1

Only the records containing **OGD** are excluded. Everything else is saved as a csvfile.

In [4]:
id_count-1

134653

There are total **134653** records in the saved csv file. Stil there are too many so I will filter the data further. First I will check the most frequently appeared **Destination** and **Origin**.

In [5]:
### Create a dictionary for Destinations and Origins
Dest_dict = {}

### Load airport_coord.csv file
with open("airport_coord.csv") as F:
    for i in csv.DictReader(F):
        ### Count Destinations
        if i["Dest"] in Dest_dict.keys():
            Dest_dict[i["Dest"]]+=1
        else:
            Dest_dict[i["Dest"]]=1

### Sort by value
import pprint
import operator
Dest_sorted = sorted(Dest_dict.items(), key=operator.itemgetter(1))

### Print out the result
print "Destinations"
pprint.pprint(Dest_sorted[-15:])

Destinations
[('MEM', 2107),
 ('MCO', 2147),
 ('LAX', 2332),
 ('LAS', 2553),
 ('EWR', 2594),
 ('CVG', 2669),
 ('PHX', 2720),
 ('SLC', 2956),
 ('MSP', 3257),
 ('IAH', 3281),
 ('DTW', 3344),
 ('DEN', 3377),
 ('DFW', 3958),
 ('ORD', 4218),
 ('ATL', 4945)]


Above is the list of top **15** destinations that will be used in the data visualization project.

In [6]:
### Save the list of top 15 airports from the array
top_Dest = []

for i in Dest_sorted[-15:]:
    top_Dest.append(i[0])

In [7]:
### open and filter the data
id_num = 0
with open("airport_coord.csv") as F:
    with open("airport_filtered.csv", "wb") as WF:
        fieldnames = ['id', 'day', 
                      'Dest', 'Dest_lon', 'Dest_lat', 
                      'Origin', 'Origin_lon', 'Origin_lat']
        writer = csv.DictWriter(WF, fieldnames=fieldnames)
        writer.writeheader()
        
        ### write rows
        tmp_line = ""
        for i in csv.DictReader(F):
            if i["Dest"] in top_Dest:
                i['id'] = id_num
                writer.writerow(i)
                id_num+=1

In [8]:
id_num-1

46457

There are now total **46457** records in the file. 

In this data visualization project, nodes and links properties in **D3.js** will be used to indicate the flight paths. So, I will parse the data to create arrays for nodes and links.

In [2]:
### Create arrays of airports in the file
airports_list = {}
count = 0
import csv
### load airport_filtered file and save nodes
with open("airport_filtered.csv") as F:
    for line in csv.DictReader(F):
        if line['Dest'] not in airports_list.keys():
            airports_list[line['Dest']] = {"ind":count, "lon": float(line['Dest_lon']), "lat":float(line['Dest_lat'])}
            count += 1
        if line['Origin'] not in airports_list.keys():
            airports_list[line['Origin']] = {"ind":count, "lon": float(line['Origin_lon']), "lat":float(line['Origin_lat'])}
            count += 1

### Sort the dictionary by ind
airports_tuple = sorted(airports_list.iteritems(), key=lambda x:x[1])

### Save the tuple as a csv file
with open("nodes.csv", "wb") as F:
    fieldnames = ['code', 'id', 'lon', 'lat'] 
    writer = csv.DictWriter(F, fieldnames=fieldnames)
    writer.writeheader()

    ### write rows
    for k in airports_tuple:
        tmp_dict = {}
        tmp_dict['code'] = k[0]
        tmp_dict['id'] = k[1]['ind']
        tmp_dict['lon'] = k[1]['lon']
        tmp_dict['lat'] = k[1]['lat']
        writer.writerow(tmp_dict)

### Save the edges as a csv file
with open("edges.csv", "wb") as F:
    fieldnames = ["source", "target", 'day']
    writer = csv.DictWriter(F, fieldnames = fieldnames)
    writer.writeheader()
    
    ### load airport_filtered and save edges
    with open("airport_filtered.csv") as csvfile:
        for line in csv.DictReader(csvfile):
            tmp_dict = {}
            tmp_dict["source"] = airports_list[line['Origin']]['ind']
            tmp_dict["target"] = airports_list[line['Dest']]['ind']
            tmp_dict["day"] = line['day']
            writer.writerow(tmp_dict)

Now the files are ready to be used in **D3.js** for data visualization. You can see the result in **final_project.html**.